In [ ]:
CON_PGSQL = {
    "HOST" : "localhost", "PORT" : "5432", "DATABASE" : "dsn_collect",
    "USER" : "postgres", "PASSWORD" : "admin", "TEMPLATE" : "postgis_template"
}

JOURNAL_TBL = {
    "TNAME"     : "jornais_web",
    "PK"        : "fid",
    "URL"       : "main_url",
    "TAG_TITLE" : "html_title",
    "TAG_CORPUS": "html_corpus",
    "CSS_CLASS" : "css_class"
}

NEWS_TBL = {
    "TNAME" : "noticias",
    "PK" : "fid",
    "URL" : "url",
    "TITLE" : "titulo",
    "WORD_GROUP" : "word_grp",
    "FK_JOURNAL" : "jornal_fid"
}

NEWS_PART = {
    "TNAME" : "noticias_parts",
    "NEWS"  : "news_fid",
    "PART"  : "news_part",
    "TEXT"  : "text"
}

SEARCH_WORDS = {
    "TNAME" : "search_words",
    "WORD"  : "word",
    "GROUP" : "group_id"
}

In [ ]:
import pandas
from gasp.fm.psql          import query_to_df
from gasp.web import get_text_in_html, get_text_in_CssClass
from gasp.cpu.pnd.mng import merge_df
from gasp.cpu.psql.mng.qw import exec_write_q

In [ ]:
"""
Add news text to the database
"""

NEWS_TO_GET = query_to_df(CON_PGSQL, (
    "SELECT main.{}, main.{}, "
    "j.{}, j.{}, j.{} "
    "FROM {} AS main INNER JOIN {} AS j "
    "ON main.{} = j.{} "
    "WHERE {} IS NULL AND jornal_fid = 1"
).format(
    NEWS_TBL["PK"]   , NEWS_TBL["URL"],
    JOURNAL_TBL["TAG_TITLE"], JOURNAL_TBL["TAG_CORPUS"],
    JOURNAL_TBL["CSS_CLASS"],
    NEWS_TBL["TNAME"], JOURNAL_TBL["TNAME"],
    NEWS_TBL["FK_JOURNAL"], JOURNAL_TBL["PK"],
    NEWS_TBL["TITLE"]
))

In [ ]:
print NEWS_TO_GET

In [ ]:
NEWS_DF = []
TITLES  = {}

In [ ]:
def _news_text(row):
    fid        = row[NEWS_TBL["PK"]]
    url        = row[NEWS_TBL["URL"]]
    tag_title  = row[JOURNAL_TBL["TAG_TITLE"]]
    tag_corpus = row[JOURNAL_TBL["TAG_CORPUS"]]
    tag_css    = row[JOURNAL_TBL["CSS_CLASS"]]
    
    title = get_text_in_html(url, tags=[tag_title])
    if not title[tag_title]:
        TITLES[int(fid)] = "NO_TITLE"
        return
    
    txtData = [get_text_in_CssClass(
        url, tag_corpus, csst
    ) for csst in tag_css]
    
    __txtData = []

    for t in txtData:
        if not t:
            continue
    
        __txtData += t['p']
    
    if not __txtData:
        TITLES[int(fid)] = "NO_TEXT"
        return
    
    new_text = pandas.DataFrame(__txtData, columns=[NEWS_PART["TEXT"]])
    new_text[NEWS_PART["NEWS"]] = fid
    new_text[NEWS_PART["PART"]] = new_text.index + 1
    
    TITLES[int(fid)] = title[tag_title][0]
    NEWS_DF.append(new_text)

In [ ]:
NEWS_TO_GET.apply(lambda x: _news_text(x), axis=1)

In [ ]:
APP_NEWS_TABLE = merge_df(NEWS_DF)

In [ ]:
APP_NEWS_TABLE[NEWS_PART["PART"]] = APP_NEWS_TABLE[NEWS_PART["PART"]].astype(int)

In [ ]:
# Update Titles

UPDATE_QUERY = unicode((
    "UPDATE {tbl} SET {titleCol} = '{titleText}' "
    "WHERE {idCol} = {idNew}"
), 'utf-8')

UPDATE_QUERIES = [UPDATE_QUERY.format(
    tbl=unicode(NEWS_TBL["TNAME"], 'utf-8'),
    titleCol=unicode(NEWS_TBL["TITLE"], 'utf-8'),
    titleText=TITLES[T].replace("'", ""),
    idCol=unicode(NEWS_TBL["PK"], 'utf-8'),
    idNew=unicode(str(T), 'utf-8')
) for T in TITLES]

exec_write_q(CON_PGSQL, UPDATE_QUERIES)

In [ ]:
# Add news text to database

from gasp.to.psql import df_to_pgsql

df_to_pgsql(CON_PGSQL, APP_NEWS_TABLE, NEWS_PART["TNAME"], append=True)